## Analyze A/B Test Results

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

**As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.** The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [ ]:
df = pd.read_csv('ab_data.csv')
df.head()

b. Use the below cell to find the number of rows in the dataset.

In [ ]:
df_rows = df.shape[0]
print(f'The number of rows in the dataframe is: {df_rows}')

c. The number of unique users in the dataset.

In [ ]:
unique_user = df.user_id.nunique()
print(f'The number of unique users in the dataframe is: {unique_user}')

d. The proportion of users converted.

    Proportion of users converteed can be found by calculating the mean or by taking sum of all value and dividing by the total users and than multiplying with 100

In [ ]:
conv_users = (df.converted.mean())*100
print(f'Proportion of converted users in the dataframe is: {conv_users} %')

e. The number of times the `new_page` and `treatment` don't line up.

In [ ]:
mismatch_values = df[((df['group'] == 'treatment') == (df['landing_page'] == 'new_page')) == False].shape[0]
print(f'The mismatching values in the dataframe is: {mismatch_values}')

f. Do any of the rows have missing values?

In [ ]:
missing_values = df.isnull().sum()
print(f'The missing values in the dataframe is: \n {missing_values}')

`2.` For the rows where **treatment** is not aligned with **new_page** or **control** is not aligned with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to provide how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [ ]:
# df2 contains only the matching values of treatment and control
df2 = df.drop(df[((df['group'] == 'treatment') == (df['landing_page'] == 'new_page')) == False].index)

In [ ]:
df2.shape

In [ ]:
df.head(3)

In [ ]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [ ]:
unique_users2 = df2.user_id.nunique()
print(f'The number of unique users in the dataframe is: {unique_users2}')

b. There is one **user_id** repeated in **df2**.  What is it?

In [ ]:
# unique user and the shape of df2 has one difference so there should be "one" user repeated
total_repeated_user_id = sum(df2['user_id'].duplicated())
print(f'The number of repeated users in the dataframe is: {total_repeated_user_id}')

c. What is the row information for the repeat **user_id**? 

In [ ]:
df2[df2.duplicated(['user_id'], keep=False)]

d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [ ]:
# keeping the latest timestamp of the user_id
df2 = df2.drop(df2.query("timestamp == '2017-01-09 05:37:58.781806'").index)

In [ ]:
# validation of dropped row
df2.info()
#290584 is the number of unique user ids

`4.` Use **df2** in the below cells to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [ ]:
prob_conv = df['converted'].mean()
print(f' The probability of an individual converting is: {prob_conv}')

b. Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
# groupby function will help sort control and treatment groups apart and calculate mean for both the groups
df_grp = df.groupby('group').mean()
df_grp

Probablity of control group is **0.120399**

c. Given that an individual was in the `treatment` group, what is the probability they converted?

Probablity of treatment group is **0.118920**

d. What is the probability that an individual received the new page?

In [ ]:
df2.landing_page.value_counts()/df2.shape[0]

e. Consider your results from a. through d. above, and explain below whether you think there is sufficient evidence to say that the new treatment page leads to more conversions.

#### **Answer**
The Answer is divided into few parts to provide proper explanation as to why those statement where made.

**1**. The probablity of old page compared to new page is 0.14% more, which is not sufficient to conclude new treatment page leads to more conversion. According to this percentage the new treatment page is not performing well compared to old control page.

In [ ]:
control_prob = 0.120399
treatment_prob = 0.118920

# Calculate the proportion difference and convert to percentage
percentage_more_control = (control_prob - treatment_prob) * 100

print(f' Percentage more on control group compared to treatment group: {percentage_more_control} %')

**2.** The Time duration the new treatement page was tested should also be taken into consideration.
A treatment group for 22 days does not seem enough data

In [ ]:
treatment_df = df[df['group'] == 'treatment']
treatment_df['timestamp'] = pd.to_datetime(treatment_df['timestamp'])

# Calculate the difference between maximum and minimum timestamps for the treatment group
time_diff = treatment_df['timestamp'].max() - treatment_df['timestamp'].min()

print("Total time for the treatment group:", time_diff)

3. There is also a factor of change aversion which could affect the adoption of new treatment page compared to old control page. And also many other factor affects the new tratement page
#### Concluding #### 
One cannot say with certainity the new treatment page leads to more conversions.

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

#### **Answer**

- **Hypothesis**

$$H_0: p_{new} \leq p_{old}$$

$$H_1: p_{new} > p_{old}$$

- **In other words**

$$H_0: p_{new} - p_{old} \leq 0$$

$$H_1: p_{new} - p_{old} > 0$$

where 
H_0 is Null hypothesis and H_1 is Alternative hypothesis


`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

a. What is the **convert rate** for $p_{new}$ under the null? 

In [ ]:
p_new = df2['converted'].mean()
print(f' Convert rate fpr p_new: {p_new} ')

b. What is the **convert rate** for $p_{old}$ under the null? <br><br>

In [ ]:
p_old = df2['converted'].mean()
print(f' Convert rate for p_old: {p_old}')

c. What is $n_{new}$?

In [ ]:
n_new = len(df2.query("group == 'treatment'"))
print(f' the number of individuals in the treatment group: {n_new}')

d. What is $n_{old}$?

In [ ]:
n_old = len(df2.query("group == 'control'"))
print(f' the number of individuals in the control group: {n_old}')

e. Simulate $n_{new}$ transactions with a convert rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [ ]:
new_page_converted = np.random.choice([1,0], size = n_new, p = [p_new, (1- p_new)])
new_page_converted.mean()

f. Simulate $n_{old}$ transactions with a convert rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [ ]:
old_page_converted = np.random.choice([1,0], size = n_old, p = [p_old, (1- p_old)])
old_page_converted.mean()

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [ ]:
print(old_page_converted.shape, new_page_converted.shape)

In [ ]:
# both dataframe are of different shape, so making both shape equal

new_page_converted = new_page_converted[:len(old_page_converted)]

h. Simulate 10,000 $p_{new}$ - $p_{old}$ values using this same process similarly to the one you calculated in parts **a. through g.** above.  Store all 10,000 values in a numpy array called **p_diffs**.

In [ ]:
p_diff = (new_page_converted/n_new) - (old_page_converted/n_old)
p_diff

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
p_diffs = []

for _ in range(10000):
    new_page_converted = np.random.choice([1,0], size = n_new, p = [p_new, (1- p_new)]).mean()
    old_page_converted = np.random.choice([1,0], size = n_old, p = [p_old, (1- p_old)]).mean()
    diff = new_page_converted - old_page_converted
    p_diffs.append(diff)

In [ ]:
p_diffs = np.array(p_diffs)

In [ ]:
p_diffs.mean()

In [ ]:
plt.hist(p_diffs);

In [ ]:
#NumPy array containing the 10,000 random samples generated from the normal distribution
nullvals = np.random.normal(0, p_diffs.std(), 10000)
plt.hist(nullvals);

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
actual_diff = df2.query("group == 'treatment'")['converted'].mean() - df2.query("group == 'control'")['converted'].mean()
print(f' Proportion of the p_diffs that are greater than the actual difference observed in csv: {actual_diff}')

In [ ]:
plt.hist(nullvals);
plt.axvline(actual_diff, color = 'red');

In [ ]:
(nullvals > actual_diff).mean()

k. In words, explain what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

#### **Answer**

- From the **p-value (0.90)** calculated in j-section, the difference in conversion rate between new page and old page does not appear to be significant. A larger p-value suggests weaker evidence against the null hypothesis. We fail to reject the null hypothesis, suggesting that there is no statistically significant evidence that the new page performs better than the old page.
- The data indicates a negative difference between the new page and old page, implying that the old page could have a slight advantage over the new page.

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
import statsmodels.api as sm

convert_old = sum(df2.query("group == 'control'")['converted'])
convert_new = sum(df2.query("group == 'treatment'")['converted'])
n_old = len(df2.query("group =='control'"))
n_new = len(df2.query("group =='treatment'"))

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](http://knowledgetack.com/python/statsmodels/proportions_ztest/) is a helpful link on using the built in.

In [ ]:
z_score, p_value = sm.stats.proportions_ztest([convert_old, convert_new], [n_old, n_new], alternative='smaller')
print(f' z_score: {z_score}, p_value:{p_value}')

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

#### **Answer**
- The P-Values calculated achieves similar results to previous calculation and leading again to the same deduction.Old page is bit better than new page and we cannot reject the null hypothesis

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you acheived in the previous A/B test can also be acheived by performing regression.<br><br>

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

### Logistic Regression

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives.  However, you first need to create a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
df2['intercept'] = 1

df2[['control', 'treatment']] = pd.get_dummies(df2['group'])
df2.head()

c. Use **statsmodels** to import your regression model.  Instantiate the model, and fit the model using the two columns you created in part **b.** to predict whether or not an individual converts.

In [ ]:
import statsmodels.api as sm
logitmod = sm.Logit(df2['converted'], df2[['intercept', 'treatment']])

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
results = logitmod.fit()
results.summary()

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in the **Part II**?

#### **Answer**

- **Hypothesis**

  $$H_0: p_{new} - p_{old} = 0$$

  $$H_1: p_{new} - p_{old} \neq 0$$

- The P-Value in **Part-2** is about whether the conversion rate for the new page is greater than the old page, implying it being one-tailored test where alternative test is directional and the P-value in **Part-3** is whether there is a difference in conversion rate between the new page and old page two tailed test where alterntative test is non-directional. That's why there is difference in P-Value for Part-2 and Part-3.
- P-Vaue = **0.19** which is greater than **0.5**, iindicating here as well we cannot reject the null hypothesis and arrives at the same results of Part-2

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

#### **Answer**

#### Considering other factors
- Adding segment of new users and returning users,change aversion and predisposition to conversion helps in optimizing user experiences and tailoring strategies to improve conversion rates for both segments
- Regionality involves accounting for geographical or regional variations in user behavior. Different countries, regions, or cultures may have distinct holidays, celebrations, or significant events that impact user engagement and conversion rates.


#### Disadvantages of considering other factors
- Multicollinearity can make it challenging to interpret the individual effects of predictors accurately and can lead to unstable coefficient estimates.
- Even after deep analyzed research what other factors to take into consideration, there would be many more influencing factors as well.

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives. You will need to read in the **countries.csv** dataset and merge together your datasets on the approporiate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [ ]:
countries_df = pd.read_csv('./countries.csv')
df_new = countries_df.set_index('user_id').join(df2.set_index('user_id'), how='inner')

In [ ]:
df_new.head(3)

In [ ]:
df_new.landing_page.value_counts()

In [ ]:
df_new.country.value_counts()

In [ ]:
### Create the necessary dummy variables
df_new[['CA', 'UK', 'US']] = pd.get_dummies(df_new['country'])
df_new.head(3)

In [ ]:
### Fit Your Linear Model And Obtain the Results
df_new['intercept'] = 1

logitmod = sm.Logit(df_new['converted'], df_new[['intercept', 'UK', 'US']])
results = logitmod.fit()
results.summary()

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [ ]:
df_new['US_treatment'] = df_new['US'] * df_new['treatment']
df_new['UK_treatment'] = df_new['UK'] * df_new['treatment']

In [ ]:
### Fit Your Linear Model And Obtain the Results

logitmod2 = sm.Logit(df_new['converted'], df_new[['intercept', 'treatment', 'UK', 'US', 'UK_treatment', 'US_treatment']])
results2 = logitmod2.fit()
results2.summary()

In [ ]:
#estimated coefficients from the regression model
np.exp(results.params)

#### Regression
- Including country and pages does not have significant affect on conversion based on P-Value, which could reject the Null Hypothesis if the value was less than **0.05**
- The Domain knowledge would play a important role in defining the factors to be included into the model so the vital parameters are included in the model and model is not overloaded with parameters that play no role in improving the conversion prediction like country

<a id='conclusions'></a>
## Conclusions
-  The performance of old page compared to new page is slighty better after computing different techniques in Part-2 and Part-3 and failing to reject the Null Hypothesis according to the interpretation of the particular techniques used to calculate the P-Value.
    - Part-2 the higher p-value of weak 0.9 indicating against the Null Hypothesis. 
    - Regression had P-Value 0.19 when it should be less than 0.05 to reject the Null Hypothesis.
 
Congratulations on completing the project! 

### Gather Submission Materials

Once you are satisfied with the status of your Notebook, you should save it in a format that will make it easy for others to read. You can use the __File -> Download as -> HTML (.html)__ menu to save your notebook as an .html file. If you are working locally and get an error about "No module name", then open a terminal and try installing the missing module using `pip install <module_name>` (don't include the "<" or ">" or any words following a period in the module name).

You will submit both your original Notebook and an HTML or PDF copy of the Notebook for review. There is no need for you to include any data files with your submission. If you made reference to other websites, books, and other resources to help you in solving tasks in the project, make sure that you document them. It is recommended that you either add a "Resources" section in a Markdown cell at the end of the Notebook report, or you can include a `readme.txt` file documenting your sources.

### Submit the Project

When you're ready, click on the "Submit Project" button to go to the project submission page. You can submit your files as a .zip archive or you can link to a GitHub repository containing your project files. If you go with GitHub, note that your submission will be a snapshot of the linked repository at time of submission. It is recommended that you keep each project in a separate repository to avoid any potential confusion: if a reviewer gets multiple folders representing multiple projects, there might be confusion regarding what project is to be evaluated.

It can take us up to a week to grade the project, but in most cases it is much faster. You will get an email once your submission has been reviewed. If you are having any problems submitting your project or wish to check on the status of your submission, please email us at dataanalyst-project@udacity.com. In the meantime, you should feel free to continue on with your learning journey by beginning the next module in the program.